## Monthly Accounting

In [19]:
# General Libraries

import os
from bs4 import BeautifulSoup
import requests
import pymongo
import re
import json
from datetime import datetime
import time

# Dataframe manipulation
from glob import glob
import pandas as pd

# Database
import psycopg2
from config import config
from sqlalchemy import create_engine

# Cryptography Library
from cryptography.fernet import Fernet

In [2]:
# Load Script Parameters

# Code References
    # https://www.scaler.com/topics/how-to-open-json-file-in-python/
    # https://bobbyhadz.com/blog/python-assign-dictionary-value-to-variable#assign-dictionary-key-value-pairs-to-variables-in-python

# Open json file with the parameters
with open("Monthly-Accounting-Params.json", "r") as file:
    jsonData = json.load(file)
# print("Datatype of variable: ", type(jsonData))

# Initiate variables
locals().update(jsonData['crypto'])
locals().update(jsonData['toggl'])
locals().update(jsonData['mail'])

# Decode passwords
key = bytes(pwd_key, 'utf-8')
fernet = Fernet(key)

mail_password = fernet.decrypt(bytes(mail_password, 'utf-8')).decode()

print(nd_mail_body)

Olá Elaine,

Em anexo a Nota de Débito do Mailchimp deste mês.

Oscar


In [3]:
# Create destination folder and/or mount the path

today = datetime.today()
current_year = today.year
current_month = today.month

# Processing year and month
month = '12' if current_month == 1 else str(current_month - 1).zfill(2)
year = str(current_year) if current_month > 1 else str(current_year - 1)
# print(month)
# print(year)

dest_folder = toggl_folder + year + '\\' + month
print(dest_folder)

if not os.path.isdir(dest_folder):
   os.mkdir(dest_folder)


D:\Primeway\Primeway\Administração\Planejamento Diário\Toggl\2023\07


In [9]:
# Connect to Google Driver
# Source: # https://stackoverflow.com/questions/66209119/automation-google-login-with-python-and-selenium-shows-this-browser-or-app-may

import undetected_chromedriver as uc

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Options
options = uc.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# prefs = {'download.default_directory' : 'D:\\Primeway\\Primeway\\Administração\\Planejamento Diário\\Toggl'}
prefs = {'download.default_directory' : dest_folder}
options.add_experimental_option('prefs', prefs)


driver = uc.Chrome(use_subprocess=True, options=options)
driver.get("https://track.toggl.com/")

In [10]:
# Get Gmail Account and Password

# email = input("Google Account:")
password = input("Google Password:")
print('Ok')

Ok


In [13]:
# Google Login

try:

   # driver.find_element(By.CSS_SELECTOR, '.css-154ozm6-SocialLogin').click()
   driver.find_element(By.CSS_SELECTOR, 'a.css-1dct05').click()

   email_field = WebDriverWait(driver,20).until(
      EC.presence_of_element_located((By.ID, 'identifierId'))
   )
   email_field.send_keys(mail_sender)
   email_field.send_keys(Keys.RETURN)
   
   driver.implicitly_wait(10)
   
   password_field = WebDriverWait(driver, 20).until(
      EC.presence_of_element_located((By.CSS_SELECTOR, 'input[name="Passwd"]'))
   )
   password_field.send_keys(password)
   password_field.send_keys(Keys.RETURN)
   
except Exception as error:
   print(error)

In [21]:
# Go to Previous Month Report Page and Download CSV File

try:

   driver.get('https://track.toggl.com/reports/detailed/1891953/period/prevMonth')

   # Detailed report - Click the download CSV link
   elem = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.CSS_SELECTOR, '.css-4jnc92-Button-button-button-bodyText-PlainButton-EnhancedPlainButton'))
   )
   elem.click()
   elem = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.CSS_SELECTOR, '.css-5drhz0-Container [data-heap-id="csv"]'))
   )
   elem.click()

except Exception as error:
   print(error)


In [4]:
# Verify if all Toggl task has an associated project

# print(dest_folder + "\\*.csv")

tasks = glob(dest_folder + "\\*.csv")[0]
df = pd.read_csv(tasks)

pd.options.display.max_rows = 999

# df

filtered_df = df[df['Project'].isnull()]

if filtered_df.size == 0:
   print("All tasks have an associated project!")
else:
   filtered_df

All tasks have an associated project!


In [ ]:
# a = df.columns.values
# # print(a)
# print(*a,sep='\n')

In [30]:
# Download PDF Files

def download_pdf():
   elem = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.CSS_SELECTOR, '.css-4jnc92-Button-button-button-bodyText-PlainButton-EnhancedPlainButton'))
   )
   elem.click()
   elem = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.CSS_SELECTOR, '.css-5drhz0-Container [data-heap-id="pdf"]'))
   )
   elem.click()


try:

   # Detailed report - Click the download PDF link
   driver.get('https://track.toggl.com/reports/detailed/1891953/period/prevMonth')
   WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'reports-header')))
   download_pdf()

   time.sleep(10)

   # Sumemary report - Click the download PDF link
   driver.get('https://track.toggl.com/reports/summary/1891953/period/prevMonth')
   WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'reports-header')))
   download_pdf()

except Exception as error:
   print(error)


In [24]:
# Connect to PostgreSQL Database

# read connection parameters
params = config()

# connect to the PostgreSQL server
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(**params)
conn.autocommit = True

# create a cursor
cursor = conn.cursor()


print('Cursor created!')

Connecting to the PostgreSQL database...
Cursor created!


In [32]:
# Save pandas dataframe to database table

# Source: 
# https://www.geeksforgeeks.org/how-to-write-pandas-dataframe-to-postgresql-table/
# https://stackoverflow.com/questions/57761954/delete-rows-from-sql-server-bases-on-content-in-dataframe

# establish connections
conn_string = "postgresql://{}:{}@{}/{}".format(params['user'], params['password'], params['host'], params['database'])
db_sql = create_engine(conn_string)
conn_sql = db_sql.connect()

params['host']

'localhost'

In [31]:
df.to_sql('time_entries', conn_sql, if_exists='append', index=False)

233

In [29]:
df

# df[["User","Email","Client","Project"]]

,User,Email,Client,Project,Task,Description,Billable,Start date,Start time,End date,End time,Duration,Tags,Amount ()
0,Oscar Carvalho,oscar@primeway.com.br,Primeway,Treinamento,NaN,Economics study,No,2023-07-01,10:05:29,2023-07-01,11:08:15,01:02:46,NaN,NaN
1,Oscar Carvalho,oscar@primeway.com.br,Primeway,Treinamento,NaN,Statistics Study,No,2023-07-01,23:12:27,2023-07-02,00:13:05,01:00:38,NaN,NaN
2,Oscar Carvalho,oscar@primeway.com.br,Primeway,Treinamento,NaN,Statistics Study,No,2023-07-02,08:13:44,2023-07-02,08:14:14,00:00:30,NaN,NaN
3,Oscar Carvalho,oscar@primeway.com.br,Primeway,Treinamento,NaN,Statistics Study,No,2023-07-02,08:40:47,2023-07-02,09:40:47,01:00:00,NaN,NaN
4,Oscar Carvalho,oscar@primeway.com.br,Primeway,Treinamento,NaN,Economics study,No,2023-07-02,10:30:25,2023-07-02,10:49:40,00:19:15,NaN,NaN
5,Oscar Carvalho,oscar@primeway.com.br,Primeway,Treinamento,NaN,Economics study,No,2023-07-02,11:04:57,2023-07-02,11:14:38,00:09:41,NaN,NaN
6,Oscar Carvalho,oscar@primeway.com.br,Primeway,Treinamento,NaN,Economics study,No,2023-07-02,11:46:15,2023-07-02,12:18:35,00:32:20,NaN,NaN
7,Oscar Carvalho,oscar@primeway.com.br,Primeway,Serviços Gerais,NaN,Financial Management,No,2023-07-02,18:08:58,2023-07-02,19:10:16,01:01:18,NaN,NaN
8,Oscar Carvalho,oscar@primeway.com.br,Primeway,Serviços Gerais,NaN,Workstation preparation,No,2023-07-02,20:41:18,2023-07-02,22:20:23,01:39:05,NaN,NaN
9,Oscar Carvalho,oscar@primeway.com.br,Primeway,Serviços Gerais,NaN,Daily Organization,No,2023-07-03,08:30:06,2023-07-03,08:47:22,00:17:16,NaN,NaN
